In [104]:
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pylab import savefig

In [2]:
# Load the clean data into variables
DataPath = "Processed Data/"

dicts = ['EFI_ID_List', 'metabolite_dict', 'Protein_seq_dict']

with open(DataPath + 'EFI_ID_List.p', 'rb') as EFI_ID:
    EFI_ID_List = pickle.load(EFI_ID)

with open(DataPath + 'metabolite_dict.p', 'rb') as metabolite:
    metabolite_dict = pickle.load(metabolite)

with open(DataPath + 'Protein_seq_dict.p', 'rb') as Protein_seq:
    Protein_seq_dict = pickle.load(Protein_seq)

activations = pd.read_csv(DataPath + 'activations.csv', index_col=0)

In [153]:
def Heatmap(df=activations, xlabel="Protein Index", ylabel="Substrate Index", dpi=400):
    # Generate heatmap of raw data and activations
    plt.figure(dpi=dpi)
    heatmap = plt.imshow(df) # Creates heatmap using input dataframe/array
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.jet() # Specifies the "jet" colormap
    plt.colorbar() # Displayes colourbar
    plt.show(heatmap)
    plt.close()

def Heatmap_Vector(df=activations.iloc[0,:], ylabel="Protein Index", xlabel="Substrate Index", my_dpi=400, show_values=True, showfig=False, savefig=True):
    # Generate heatmap from data contained in a vector using seaborn
    fig, ax = plt.subplots(figsize=(0.5,40), dpi=my_dpi)

    sns.heatmap(data=df,
                cmap='jet',
                linewidths=0,
                annot=show_values,
                yticklabels=True,
                cbar_kws={"shrink":10.0, "anchor":(0.0, 1.0)},
                ax=ax)
    
    
    ax.set(title='Ranked Protein Activity',
            xlabel=xlabel,
            ylabel=ylabel)
    
    if showfig == True:
        plt.show(ax)

    if savefig == True:
        path = "Heatmap_Vector_Plots/"
        plt.savefig(path + str(xlabel) + ".png", dpi=my_dpi, bbox_inches='tight')

    plt.close(fig)

def SortNormalize(df):
    # Sort values in dataframe, normalize data, retain labels on protein ID
    df = df.sort_values(by=SID, ascending=False) # Sort values in descending order
    # Still need to add data normalization feature
    print(df[df[SID] > 0].count()) # Print count of non-zero datapoints
    return df

In [154]:
def SaveSubstrate(df=activations, SID=0, Subs_list=list(metabolite_dict.values())):
    df = pd.DataFrame(df.iloc[SID,:]) # Obtains vector from dataframe based on substrate needed for analysis

    df = SortNormalize(df=df)
    substrate = Subs_list[SID]

    Heatmap_Vector(df=df, show_values=False, xlabel=substrate, savefig=True)

'''
SID = np.random.randint(0,168)
print(SID)
AnalyzeSubstrate(df=activations, SID=SID)
'''

164
164    103
dtype: int64


In [ ]:
for i in range(0,len(activations.index)):
    AnalyzeSubstrate(df=activations, SID=i)
    



In [43]:
path = "Heatmap_Vector_Plots/"
print(path + str(list(metabolite_dict.values())[0]) + '.png')

Heatmap_Vector_Plots/Blank.png
